In [2]:
import numpy as np
import circle_fit as cf
import os
import cv2
%run Functions.ipynb

## 重複過半的 Pixel 才拿來用

In [3]:
def Get_ROI_Loc(cy,cx,cr):
    ## Loc 從左上到右下，y放前面（符合 image 讀取）
    x, y, r = int(round(cx)), int(round(cy)), int(round(cr))
    roi_loc = [[x-r-10, y-r-10], [x+r+10, y+r+10] ]
    return np.asarray(roi_loc)

In [4]:
def Get_ROI(image):
    ## 拿一張影像先算 Center 拿來取 ROI
    _, _, _, edge = image_processing(image)
    edge_pts = get_edge_points(edge)
    cx,cy,cr,_ = cf.least_squares_circle((edge_pts))
    
    roi_loc = Get_ROI_Loc(cy,cx,cr)
    return roi_loc

In [39]:
def Analize_Multi_Frames(roi_loc, images):
    ## 取得所有當批影像的 ROI 
    lx, ly = roi_loc[0,:]
    rx, ry = roi_loc[1,:]
    imgs_crop = [image[ly:ry, lx:rx] for image in images]
    
    
    ## 取得每張邊緣影像的
    imgs_edge = []
    for cropped in imgs_crop:
        _, _, _, edge = image_processing(cropped)
        imgs_edge.append(edge)
    
    
    ## 取得每張邊緣的點
    imgs_edge_pts = [get_edge_points(edge) for edge in imgs_edge]
    
    
    ## 產生 Map 來紀錄出現次數
    Map = np.zeros(images[0].shape)
    for img_edge_pts in imgs_edge_pts:
        for x, y in img_edge_pts:
            Map[y,x] = Map[y,x] + 1
   
    
    # 塞選出重複出現一半次數的點
    valid_pts = []
    for i, row in enumerate(Map):
        for j, val in enumerate(row):
            if(val >= 5):
                valid_pts.append([j,i])
    
    cx, cy, r, _ = cf.least_squares_circle((valid_pts))
    predicted = [cx+lx, cy+ly] 
    return predicted

## 測試前檢查

In [17]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Cut")
files = os.listdir()
files.sort()
files = np.asarray(files)
batches = np.split(files, 20)

img = cv2.imread(files[0], 0)
roi_loc = Get_ROI(img)

lx, ly = roi_loc[0,:]
rx, ry = roi_loc[1,:]
imgs_crop = img[ly:ry, lx:rx]

blur, nlm, otsu, edge = image_processing(imgs_crop)

cv2.imshow("Blur", blur)
cv2.imshow("Nlm", nlm)
cv2.imshow("Otsu", otsu)
cv2.imshow("Edge", edge)
cv2.waitKey(0)
cv2.destroyAllWindows()

## 實際測試

In [40]:
os.chdir("/home/mj/HardDisk/Github/NTU/ARCS/img/X-Ray/Case_1/Ori_Cut")

files = os.listdir()
files.sort()
files = np.asarray(files)
batches = np.split(files, 10)

predictions = []
for batch in batches:
    images = [cv2.imread(img,0) for img in batch]
    roi_loc = Get_ROI(images[0])
    predicted = Analize_Multi_Frames(roi_loc, images)
    predictions.append(predicted)
    print("{}, {}".format(predicted[0], predicted[1]))

134.50333398625185, 98.10998888263049
134.50959172838603, 98.16942205172815
134.53809127740683, 98.03368872765179
134.59966855983106, 97.98431229201236
134.54745604253853, 98.10317354024164
134.55085150570304, 98.04816078430389
134.63551078200146, 98.24284936573301
134.544234945626, 98.21229297585452
134.51673596745792, 98.09486677001739
134.55078319712635, 98.07903283921495


In [41]:
show_resoult(predictions)

-----------------------------------------------------
Statistics: 
Mean Centroid: (134.54963, 98.10778)
Variance     : (0.00149, 0.00583)
Max Length   : 0.13218, 0.25854 (pixels)


In [ ]:
Max Length   : 0.16139, 0.22641 (pixels)

## 算機率

In [22]:
from scipy.special import comb

def Count_Probability(num_tal, num_sel, prob):
    x = prob
    y = 1 - x
    
    result = 0
    for i in range(num_sel, num_tal+1):
        num_x = i 
        num_y = num_tal - i
        result = result + comb(num_tal, i) * x**num_x * y**num_y
    
    return result

Count_Probability(20,10,0.65)

0.9468333856370541